## pymatgen

pymatgenはmaterials projectで作成された物質を扱うPythonライブラリです．

pymatgenの簡単な紹介を行います．

### 原子座標


In [ ]:
import sys
import os
from pymatgen.core import Structure
from pymatgen.io.cif import CifWriter
from pymatgen.io.xcrysden import XSF


def fileconvert2cif(file1, file2):
    """
    file format conversion
    ref.s
    http://pymatgen.org/usage.html

    @param file1 : input filename, any formats pymatgen accepts
    @param file2 : cif output filename
    """
    tmpfilename = "__temp__.cif"

    def write2cif(f1, f2):
        structure = Structure.from_file(f1)
        w = CifWriter(structure)
        w.write_file(f2)

        return structure

    def write2xsf(f1, f2):
        structure = Structure.from_file(f1)

        xsf = XSF(structure)
        s = xsf.to_string()
        with open(f2, "wb") as f:
            f.write(s)

        return structure
    _ = write2cif(file1, tmpfilename)
    """
    run twice to make atoms inside the cell automatically.
    """
    structure = write2cif(tmpfilename, file2)

    os.remove(tmpfilename)
    return structure


structure = fileconvert2cif("../data/Fe4_xsf/hcp.xsf", "tmp1.cif")


In [ ]:
structure

In [ ]:
print(len(structure))
for s in structure:
    print(s)

In [ ]:
def show_neighbors(structure):
    rcut = 3.0
    for s in structure:
        print("center", s.coords, s.specie)
        nn = structure.get_neighbors(s, rcut)
        print("number of atoms", len(nn))
        for n in nn:
            print("cart", n[0].coords, n[0].specie, "distance", n[1])


show_neighbors(structure)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


def plot_rdf(structure, rcut):
    """
    plot radial distribution function (rdf) with the distance list given by pymatgen library

    @param structure : pymatgen structure
    @param  rcut : cutoff distance
    """
    distancelist = []
    for s in structure:
        nn = structure.get_neighbors(s, rcut)
        for n in nn:
            distancelist.append(n[1])

    def plot_hist(distancelist, bins=50):
        """
        histgram plot

        @param distancelist : a list of distances
        @param bins : a list of bins
        """
        plt.xlabel("r")
        plt.ylabel("occurence")
        """
        histgram library of matplotlib
        """
        plt.hist(distancelist, bins=bins)
        plt.show()

    plot_hist(distancelist)

    def plot_rdf_div_r2(distancelist, rcut, xmin=1.0, bins=50):
        """
        multiply histgram by 1/r**2
        and plot it

        @param distancelist : a list of distance
        @param xrange : 1d array [2], min and max
        @param bins : number of bins of histgram
        """
        bins = np.linspace(xmin, rcut, bins)
        print(bins)
        index = np.digitize(distancelist, bins)
        """
        add to hist array
        """
        hist = np.zeros((len(bins)))
        for i, d in zip(index, distancelist):
            if i > 0 and i < len(bins):
                if bins[i-1] < d and d < bins[i]:
                    hist[i] += 1.0/d**2
                    continue
            print("warning i", i, d)
        """
        visualization
        """
        plt.xlabel("r")
        plt.ylabel("occurence/r**2")
        """
        use the middle point for x points
        """
        bins_mid = (bins[0:-1]+bins[1:])*0.5

        plt.plot(bins_mid, hist[1:])
        plt.show()

    plot_rdf_div_r2(distancelist, rcut)


plot_rdf(structure, rcut=7.0)



### 分子式

分子式の解釈,元素の説明変数取得ができます．

In [ ]:
from pymatgen.core.composition import Composition
comp = Composition("(FeO3Cl)Cl3.5")
print("str(comp)=", str(comp))
print("comp.num_atoms=", comp.num_atoms)
print("comp.as_dict()=", comp.as_dict())
print("comp.weight=", comp.weight)



### 元素

元素の物性値を取得することができます．

In [ ]:
from pymatgen.core.periodic_table import Element
elm = Element("Fe")
print("elm.atomic_radius_calculated=", elm.atomic_radius_calculated)
print("elm.liquid_range=", elm.liquid_range)
print("elm.X(Pauling electronegativity)=", elm.X)


最後にcompsitionから各原元素を得てその元素の特性を表示する例を示します．

In [ ]:
comp = Composition("(Fe(CN)6)(CH3)3")
for key in comp:
    elm = Element(key)
    print("element={}, fraction={}, density_of_solid={}, molar_volume={}".format(
        key, comp[key], elm.density_of_solid, elm.molar_volume))
